In [17]:
using Rays

In [18]:
using LinearAlgebra: normalize!

# Define a camera
camera = Rays.Camera()
camera.screen_res .= [500, 500]

# Let the camera look towards the origin
from = Float32[2.0, 2.0, 2.0]
to = zeros(Float32, 3)
Rays.look_at!(camera, from, to)

# Define a cube at the origin
cube = Rays.Cube(zeros(Float32, 3), 1.0f0)

# Color data
data_variables = [:dim]
julia_green = Float32[0.22, 0.596, 0.149]
julia_purple = Float32[0.584, 0.345, 0.698]
julia_red = Float32[0.796, 0.235, 0.2]
julia_colors = hcat(julia_green, julia_purple, julia_red)
color = Rays.get_canvas(camera; color = true)

function get_render(dist, θ, ϕ)::Array{<:AbstractFloat, 3}

    # Adjust camera
    Rays.look_at!(camera, zeros(Float32,3), dist, θ, ϕ)

    # Get intersection data
    intersection_data = Rays.shape_view(camera, cube; data_variables)

    # Get grayscale image
    canvas_grayscale = Rays.cam_is_source(intersection_data.t);

    # Apply color to the image
    color .= 0.0
    Rays.add_color!(color, julia_colors, intersection_data.dim)
    canvas_color = Rays.apply_color(canvas_grayscale, color);

    return canvas_color
end 

get_render (generic function with 1 method)

# Using SimpleDirectMediaLayer

In [19]:
import SimpleDirectMediaLayer as SDL
using SDL2_jll

function Base.convert(::Type{SDL.LibSDL2.SDL_bool}, bool::Bool)
    return bool ? SDL.SDL_TRUE : SDL.SDL_FALSE
end

In [20]:
function set_render!(renderer, dist, θ, ϕ)::Nothing
    render = convert.(UInt8, round.(255 .* get_render(dist, θ, ϕ)))
    render = permutedims(render, [1,3,2])
    depth = 24
    pitch = sizeof(eltype(render)) * 3 * size(render)[2]
    surface = SDL.SDL_CreateRGBSurfaceFrom(
        render, 
        camera.screen_res..., 
        depth, pitch, 
        0xFF0000, 0x00FF00, 0x0000FF, 0)
    texture = SDL.SDL_CreateTextureFromSurface(renderer, surface)
    SDL.SDL_RenderCopy(renderer, texture, SDL.C_NULL, SDL.C_NULL)
    SDL.SDL_DestroyTexture(texture)
    SDL.SDL_RenderPresent(renderer)
    return nothing
end

set_render! (generic function with 1 method)

In [21]:
mutable struct Dtimer
    t::Float64
    Dtimer() = new(0.0)
end

function start!(dtimer::Dtimer)::Nothing
    dtimer.t = time_ns()/1e9
    return nothing
end

function get_Δt!(dtimer::Dtimer)::Float64
    t = time_ns()/1e9
    Δt = t - dtimer.t
    dtimer.t = t
    return Δt
end

get_Δt! (generic function with 1 method)

In [22]:
function main()::Nothing
    SDL.SDL_Init(SDL.SDL_INIT_VIDEO)

    window_name = "Cube viewer"
    window_resizable = false

    window = SDL.SDL_CreateWindow(
        window_name, 
        SDL.SDL_WINDOWPOS_CENTERED, 
        SDL.SDL_WINDOWPOS_CENTERED, 
        camera.screen_res...,
        SDL.SDL_WINDOW_SHOWN)
    SDL.SDL_SetWindowResizable(window, window_resizable)

    renderer = SDL.SDL_CreateRenderer(window, -1, 
        SDL.SDL_RENDERER_ACCELERATED | SDL.SDL_RENDERER_PRESENTVSYNC)

    θ = π/4
    ϕ = 3π/8
    dθ = 0.0
    dϕ = 0.0
    dist = 10.0
    close = false
    dtimer = Dtimer()

    set_render!(renderer, dist, θ, ϕ)
    start!(dtimer)

    try
        while !close
            event_ref = Ref{SDL.SDL_Event}()
            Δt = get_Δt!(dtimer)
            while Bool(SDL.SDL_PollEvent(event_ref))
                event = event_ref[]
                event_type = event.type
                if event_type == SDL.SDL_QUIT
                    close = true
                    break
                elseif event_type == SDL.SDL_KEYDOWN
                    scan_code = event.key.keysym.scancode 
                    if scan_code == SDL.SDL_SCANCODE_RIGHT
                        dθ += 0.5*Δt
                    elseif scan_code == SDL.SDL_SCANCODE_LEFT
                        dθ -= 0.5*Δt
                    end
                end
            end
            θ += dθ * Δt
            ϕ += dϕ * Δt
            set_render!(renderer, dist, θ, ϕ)
        end
    catch y
        println(y)
    finally
        SDL.SDL_DestroyRenderer(renderer)
        SDL.SDL_DestroyWindow(window)
        SDL.SDL_Quit()
    end
    return nothing
end

main (generic function with 1 method)

In [23]:
main()